In [159]:
import random
import numpy as np
from statistics import mean
from random import randrange
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import time
import sys
from scipy.spatial import distance
from operator import itemgetter
%matplotlib inline

In [160]:
def new_new_cross_val_score(X, y, distances, cv=5, k=5, p=2):
    fold_size = int(y.shape[0]/cv)
    indices = np.arange(y.shape[0])
    np.random.RandomState(123).shuffle(indices)
    result = np.array([])
    for i in range(cv):
        validation_idx = indices[i*fold_size:(i+1)*fold_size]
        train_idx = np.concatenate((indices[:i*fold_size], indices[(i+1)*fold_size:]))
        
        y_indices = np.argsort(distances[validation_idx][:,train_idx])
        k_closests = y[train_idx][y_indices[:,:k]].astype(int)
        bincount = np.array([np.count_nonzero(k_closests == j, axis = 1) for j in range(10)]).transpose()
        
        #random selection
        max_bin_values = np.amax(bincount,axis=1)
        a = np.array([bincount[j] == max_bin_values[j] for j in range(max_bin_values.shape[0])])
        y_pred = np.array([])
        for row in a:
            tmp = np.asarray(np.nonzero(row)).reshape(-1)
            y_pred = np.append(y_pred,np.random.choice(tmp,1,replace=False))      
#         y_pred = np.argmax(bincount,axis = 1)
        
        valid_acc = np.sum(y_pred == y[validation_idx])
        result = np.append(result,valid_acc/validation_idx.shape[0])
    
    return result

In [161]:
train = np.genfromtxt(
        'datasets/MNIST_train.csv', delimiter=',')

In [162]:
X_train, y_train = train[:,1:], train[:,0]

In [9]:
def compute_distance(test_X, train_X):
    return np.sqrt(np.sum(test_X**2, axis=1, keepdims=True) + np.sum(train_X**2, axis=1, keepdims=True).T - 2*np.dot(test_X, train_X.T))

def helper_1(train_X, train_y, cv=5, k=5):
    fold_size = int(train_y.shape[0]/cv)
    indices = np.arange(train_y.shape[0])
    np.random.RandomState(123).shuffle(indices)
    results = []
    for i in range(cv):
        validation_idx = indices[i*fold_size:(i+1)*fold_size]
        train_idx = np.concatenate((indices[:i*fold_size], indices[(i+1)*fold_size:]))
        batch_train_idx = np.array_split(train_idx, 20)
        validation_y_pred = []
        for idx in validation_idx:
            level = []
            for j in range(20):
                batch_idx = batch_train_idx[j]
                distances = compute_distance(train_X[idx].reshape(1,-1), train_X[batch_idx])
                validation_y_index = batch_idx[np.argsort(distances[0])]
                dist_idx = list(zip(distances[0][np.argsort(distances[0])][:k], train_y[validation_y_index[:k]].astype(int)))
                level = level + dist_idx
            level.sort(key=itemgetter(0))
            k_closests = [x[1] for x in level[:k]]
            validation_y_pred.append(np.argmax(np.bincount(k_closests)))
        validation_acc = np.sum(validation_y_pred == train_y[validation_idx])
        results.append(validation_acc/validation_idx.shape[0])
    return results

In [22]:
def helper_2(train_X, train_y, cv=5, k=5):
    fold_size, batch_size = int(train_y.shape[0]/cv), 20
    indices = np.arange(train_y.shape[0])
    np.random.RandomState(123).shuffle(indices)
    results = []
    for i in range(cv):
        validation_idx = indices[i*fold_size:(i+1)*fold_size]
        train_idx = np.concatenate((indices[:i*fold_size], indices[(i+1)*fold_size:]))
        batch_train_idx = np.array_split(train_idx, batch_size)
        batch_validation_idx = np.array_split(validation_idx, batch_size)
        validation_y_pred = []
        for u in range(batch_size):
            validation_unit_batch_idx = batch_validation_idx[u]
            for idx in validation_unit_batch_idx:
                level = []
                for j in range(batch_size):
                    train_unit_batch_idx = batch_train_idx[j]
                    distances = compute_distance(train_X[idx].reshape(1,-1), train_X[train_unit_batch_idx])
                    validation_y_index = train_unit_batch_idx[np.argsort(distances[0])]
                    #zip 2 array with length k
                    dist_idx = list(zip(distances[0][np.argsort(distances[0])], train_y[validation_y_index[:k]].astype(int)))
                    level = level + dist_idx
                level.sort(key=itemgetter(0))
                k_closests = [x[1] for x in level[:k]]
                validation_y_pred.append(np.argmax(np.bincount(k_closests)))
        validation_acc = np.sum(validation_y_pred == train_y[validation_idx])
        results.append(validation_acc/validation_idx.shape[0])
    return results

In [89]:
l = []
X1 = np.arange(9,0,-1).reshape(9,-1)
Y1 = np.arange(9,18).reshape(9,-1)
l += list(zip(X1,Y1))
# l.sort(key = itemgetter(0))
# print(l)
print(l[:3])

[(array([9]), array([9])), (array([8]), array([10])), (array([7]), array([11]))]


[3 1 7]
[5 4 2]


In [165]:
def helper_3(train_X, train_y, cv=5, k=5):
    fold_size, batch_numbers = int(train_y.shape[0]/cv), 20
    indices = np.arange(train_y.shape[0])
    np.random.RandomState(123).shuffle(indices)
    results = []
    for i in range(cv):
        validation_idx = indices[i*fold_size:(i+1)*fold_size]
        train_idx = np.concatenate((indices[:i*fold_size], indices[(i+1)*fold_size:]))
        batch_train_idx = np.array_split(train_idx, batch_numbers)
        batch_validation_idx = np.array_split(validation_idx, batch_numbers)
        validation_y_pred = []
        print("start:",i,"cross validation")
        for u in range(batch_numbers):
            print("start:",u,"validation batch")
            validation_unit_batch_idx = batch_validation_idx[u]
            level = [[] for i in range(validation_unit_batch_idx.shape[0])]
            for j in range(batch_numbers):
                train_unit_batch_idx = batch_train_idx[j]
                distances=distance.cdist(train_X[validation_unit_batch_idx], train_X[train_unit_batch_idx],'minkowski',2)
                #validation_y_indices size:(validation_batch_size*train_batch_size)
                k_indices = np.argsort(distances)[:,:k]
                k_distances = np.array([distances[i,indices] for i, indices in zip(range(k_indices.shape[0]),k_indices)])
                validation_y_indices = np.array([train_unit_batch_idx[indices] for indices in k_indices])
                k_labels = np.array([train_y[indices] for indices in validation_y_indices]).astype(int)
                
                #k distances size:(validation_batch_size*k)
                #k labels size:(validation_batch_size*k)                
                # level element tuple size(k,k)
                for i in range(validation_unit_batch_idx.shape[0]):
                    level[i].extend(list(zip(k_distances[i],k_labels[i])))
            
            for i in range(validation_unit_batch_idx.shape[0]):
                level[i].sort(key=itemgetter(0))
                k_closests = [x[1] for x in level[i][:k]]            
                validation_y_pred.append(np.argmax(np.bincount(k_closests)))
           
        validation_acc = np.sum(validation_y_pred == train_y[validation_idx])
        results.append(validation_acc/validation_idx.shape[0])
    return results

In [166]:
start = time.time()
acc = helper_3(X_train, y_train, cv=5, k=5)
print(f'Time spent:{time.time() - start}')
print(acc)

start: 0 cross validation
start: 0 validation batch
start: 1 validation batch
start: 2 validation batch
start: 3 validation batch
start: 4 validation batch
start: 5 validation batch
start: 6 validation batch
start: 7 validation batch
start: 8 validation batch
start: 9 validation batch
start: 10 validation batch
start: 11 validation batch
start: 12 validation batch
start: 13 validation batch
start: 14 validation batch
start: 15 validation batch
start: 16 validation batch
start: 17 validation batch
start: 18 validation batch
start: 19 validation batch
start: 1 cross validation
start: 0 validation batch
start: 1 validation batch
start: 2 validation batch
start: 3 validation batch
start: 4 validation batch
start: 5 validation batch
start: 6 validation batch
start: 7 validation batch
start: 8 validation batch
start: 9 validation batch
start: 10 validation batch
start: 11 validation batch
start: 12 validation batch
start: 13 validation batch
start: 14 validation batch
start: 15 validation ba

In [10]:
start = time.time()
acc = helper_1(X_train, y_train, cv=5, k=5)
print(f'Time spent:{time.time() - start}')
print(acc)

Time spent:18.081727027893066
[0.9083333333333333, 0.925, 0.9183333333333333, 0.9066666666666666, 0.9066666666666666]
